In [1]:
import googlemaps
from dotenv import load_dotenv
import os
import ast
import numpy as np
from geopy.distance import geodesic
import pandas as pd
import geopandas as gpd
import time
from pathlib import Path
from geopy.geocoders import Nominatim
import re



In [2]:

PROJECT_ROOT = Path().absolute().parent
DATA_DIR = PROJECT_ROOT / 'data'
RAW_DATA_DIR = DATA_DIR / 'raw'
PROCESSED_DATA_DIR = DATA_DIR / 'processed'
TEMP_DATA_DIR = DATA_DIR / 'temp'

#load in data 
sampled_clusters_gdf=gpd.read_file(PROCESSED_DATA_DIR/'sampled_clusters.gpkg')
print(sampled_clusters_gdf.station_name.value_counts(dropna=False))
#there are non-unique grid ids. 
print(sampled_clusters_gdf.grid_id.is_unique)

sampled_clusters_gdf


station_name
Aisa FM        70
Dwanwana FM    70
Dokolo FM      70
Name: count, dtype: int64
True


,grid_id,station_name,population_count,household_count,buffer_km,cluster_type,psu_prob,psu_explanation,ssu_prob,ssu_explanation,centroid_lon,centroid_lat,geometry
0,21965,Aisa FM,412.028381,84.087425,25.0,main,0.045243,1km by 1km grid cells in 25 km radio station r...,0.142709,"second stage unit=households, inclusion probab...",33.784595,1.242797,"POLYGON ((33.78909 1.23827, 33.78909 1.24732, ..."
1,22136,Aisa FM,867.115967,176.962442,25.0,main,0.095213,1km by 1km grid cells in 25 km radio station r...,0.067811,"second stage unit=households, inclusion probab...",33.793586,1.251841,"POLYGON ((33.79808 1.24732, 33.79808 1.25636, ..."
2,21627,Aisa FM,1342.766846,274.034050,25.0,replacement,0.147442,1km by 1km grid cells in 25 km radio station r...,0.043790,"second stage unit=households, inclusion probab...",33.766624,1.260896,"POLYGON ((33.77112 1.25637, 33.77112 1.26542, ..."
3,20438,Aisa FM,1109.628174,226.454729,25.0,main,0.121842,1km by 1km grid cells in 25 km radio station r...,0.052991,"second stage unit=households, inclusion probab...",33.703710,1.269960,"POLYGON ((33.7082 1.26544, 33.7082 1.27448, 33..."
4,21119,Aisa FM,802.255127,163.725536,25.0,replacement,0.088091,1km by 1km grid cells in 25 km radio station r...,0.073293,"second stage unit=households, inclusion probab...",33.739665,1.278996,"POLYGON ((33.74416 1.27447, 33.74416 1.28352, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,9481,Dokolo FM,488.350555,99.663379,25.0,replacement,0.074282,1km by 1km grid cells in 25 km radio station r...,0.120405,"second stage unit=households, inclusion probab...",33.119498,2.111445,"POLYGON ((33.12399 2.10692, 33.12399 2.11597, ..."
206,9312,Dokolo FM,304.447693,62.132182,25.0,replacement,0.046309,1km by 1km grid cells in 25 km radio station r...,0.193137,"second stage unit=households, inclusion probab...",33.110505,2.120492,"POLYGON ((33.115 2.11597, 33.115 2.12502, 33.1..."
207,11348,Dokolo FM,424.834106,86.700838,25.0,replacement,0.064621,1km by 1km grid cells in 25 km radio station r...,0.138407,"second stage unit=households, inclusion probab...",33.218414,2.084292,"POLYGON ((33.22291 2.07977, 33.22291 2.08882, ..."
208,11690,Dokolo FM,219.438843,44.783437,25.0,main,0.033378,1km by 1km grid cells in 25 km radio station r...,0.267956,"second stage unit=households, inclusion probab...",33.236402,2.102384,"POLYGON ((33.2409 2.09786, 33.2409 2.10691, 33..."


In [3]:

#split this up in batches, dwanwana dokolo first then aisa fm. 

dwanwana_dokolo_sampled_clusters_gdf=sampled_clusters_gdf.loc[sampled_clusters_gdf['station_name'].isin(['Dwanwana FM', 'Dokolo FM'])]
aisa_sampled_clusters_gdf=sampled_clusters_gdf.loc[sampled_clusters_gdf['station_name']=='Aisa FM']
#['Aisa FM', 'Dwanwana FM', 'Dokolo FM']
# Load API key
load_dotenv()
api_key = os.getenv('GOOGLE_MAPS_API_KEY')
gmaps = googlemaps.Client(key=api_key)

def get_points_on_circle(center_lat, center_lon, radius_meters, num_points=4):
    """Generate points on a circle around the center point."""
    radius_deg = radius_meters / 111000 #convert degrees to meters (1 degree ~ 111000 meters)
    
    points = []
    for i in range(num_points):
        angle = 2 * np.pi * i / num_points # create evenly spaced points, calculate the angle where to place te point, circle is 2 pi radians (360 degrees), divide by number of points to evenly space out. 
        lat = center_lat + radius_deg * np.cos(angle) #cosine gives north south position (constant across globe-->latitude)
        lon = center_lon + radius_deg * np.sin(angle) / np.cos(np.radians(center_lat)) # longitude east west component, adjust for curvature by --> dividing by cosine latitude as east west 'shrinks'  at higher latitudes (and we'd otherwise draw an oval)
        points.append((lat, lon))
    
    return points

def find_nearest_road(center_lat, center_lon, gmaps_client, verbose=False):
    """
    Find the nearest road point to a given centroid.
    Stops searching as soon as a road is found in the first radius.
    """
    try:
        # Define search radii
        search_radii = [25, 50, 100, 150, 200] #define search radii, draw a circle around the centroid. 
        
        if verbose:
            print(f"\nOriginal centroid (lat, lon): {center_lat}, {center_lon}") 
        
        closest_point = None
        min_distance = float('inf')
        
        # Search at increasing radii
        for radius in search_radii:
            if verbose:
                print(f"\nSearching at {radius} meters radius...")
            
            num_points = 4 if radius <= 50 else 8 #get 8 points on the circle
            test_points = get_points_on_circle(center_lat, center_lon, radius_meters=radius, num_points=num_points) 
            
            road_found = False  # Flag for finding any road, start at fale. 
            
            for i, origin in enumerate(test_points):
                if verbose:
                    print(f"Trying direction search {i+1}/{num_points} at {radius}m radius")
                
                destination = (center_lat, center_lon)
                
                # Get directions response object from the API. 
                directions_result = gmaps_client.directions(
                    origin=origin,
                    destination=destination,
                    mode="driving"
                )
                print(directions_result)
                if directions_result:
                    steps = directions_result[0]['legs'][0]['steps'] #api gives directions in results and steps as json object/dictionary
                    ##[{'bounds': {'northeast': {'lat': 2.120286, 'lng': 33.2175956}, 'southwest': {'lat': 2.120286, 'lng': 33.2175956}}, 'copyrights': 'Map data ©2024', 'legs': [{'distance': {'text': '1 m', 'value': 0}, 'duration': {'text': '1 min', 'value': 0}, 'end_address': 'Unnamed Road, Uganda', 'end_location': {'lat': 2.120286, 'lng': 33.2175956}, 'start_address': 'Unnamed Road, Uganda', 'start_location': {'lat': 2.120286, 'lng': 33.2175956}, 'steps': [{'distance': {'text': '1 m', 'value': 0}, 'duration': {'text': '1 min', 'value': 0}, 'end_location': {'lat': 2.120286, 'lng': 33.2175956}, 'html_instructions': 'Head', 'polyline': {'points': 'yb}K_yviE'}, 'start_location': {'lat': 2.120286, 'lng': 33.2175956}, 'travel_mode': 'DRIVING'}], 'traffic_speed_entry': [], 'via_waypoint': []}], 'overview_polyline': {'points': 'yb}K_yviE'}, 'summary': '', 'warnings': [], 'waypoint_order': []}]

                    last_step = steps[-1] #take the last step capture the last point before moving off the road. 
                    
                    road_lat = last_step['start_location']['lat'] #capture the coordinates of the point where the directions leave the road (last step), latitude
                    road_lon = last_step['start_location']['lng'] #and longitude
                    #calculate the distance between 2 points using geodesic, to establish what the closest 'leave the road point'  is to the centroid. 
                    distance = geodesic(
                        (center_lat, center_lon), 
                        (road_lat, road_lon)
                    ).meters
                    
                    if verbose:
                        print(f"Found road point at distance: {distance:.1f}m")
                    
                    road_found = True
                    
                    if distance < min_distance: #get the minumum distance if so, get the road lat and lon. 
                        min_distance = distance
                        closest_point = {
                            'lat': road_lat,
                            'lon': road_lon,
                            'distance': distance,
                            'found_at_radius': radius
                        }
            
            # If we found any road in this radius, stop searching
            if road_found and radius == 25:  # Only stop if we found a road in the first radius
                if verbose:
                    print(f"\nFound road within first radius ({radius}m), stopping search")
                break
        
        return closest_point
                
    except Exception as e:
        if verbose:
            print(f"Error: {e}")
        return None

In [4]:
def process_all_clusters(sampled_clusters_gdf):
    """
    Process all clusters to find nearest roads for their centroids.
    Returns a DataFrame with results.
    """
    # Initialize results list
    results = []
    
    # Total number of clusters for progress tracking
    total_clusters = len(sampled_clusters_gdf)
    print(f"Starting to process {total_clusters} clusters...")
    
    for idx, row in sampled_clusters_gdf.iterrows():
        print(f"\nProcessing {idx+1}/{total_clusters}: grid_id {row.grid_id}")
        
        try:
            # Get centroid coordinates
            center_lat, center_lon = row.centroid_lat, row.centroid_lon
            
            # Find nearest road
            closest_point = find_nearest_road(center_lat, center_lon, gmaps, verbose=True)
            
            # Store results
            result = {
                'grid_id': row.grid_id,
                'station_name': row.station_name,
                'buffer_km':row.buffer_km,
                'est_population_2020':row.population_count,
                'centroid_lat': center_lat,
                'centroid_lon': center_lon,
                'centroid_maps_link': f"https://www.google.com/maps?q={center_lat},{center_lon}",
                'cluster_type': row.cluster_type,
                'nearest_road_lat': closest_point['lat'] if closest_point else None,
                'nearest_road_lon': closest_point['lon'] if closest_point else None,
                'distance_to_road': closest_point['distance'] if closest_point else None,
                'nearest_road_maps_link': (f"https://www.google.com/maps?q={closest_point['lat']},{closest_point['lon']}" 
                                         if closest_point else None),
                'found_at_radius': closest_point['found_at_radius'] if closest_point else None,
                'processing_success': True if closest_point else False,
                'geometry_grid_cell': row.geometry.wkt  
            }
            
            results.append(result)
            
            # small delay to avoid hitting API limits/overflooding API.
            time.sleep(0.1)
            
        except Exception as e:
            print(f"Error processing grid_id {row.grid_id}: {e}") #check for weird stuff happening. 
            # store error result
            result = {
                'grid_id': row.grid_id,
                'station_name': row.station_name,
                'buffer_km':row.buffer_km,
                'est_population_2020':row.population_count,
                'centroid_lat': center_lat,
                'centroid_lon': center_lon,
                'centroid_maps_link': f"https://www.google.com/maps?q={center_lat},{center_lon}",
                'cluster_type': row.cluster_type,
                'nearest_road_lat': None,
                'nearest_road_lon': None,
                'distance_to_road': None,
                'nearest_road_maps_link': None,
                'found_at_radius': None,
                'processing_success': False,
                'geometry_grid_cell': row.geometry.wkt  
            }
            results.append(result)
    
    # Convert results to DataFrame and return
    return pd.DataFrame(results)

# API calls dwanwana_dokolo:
results_dwanwana_dokolo = process_all_clusters(dwanwana_dokolo_sampled_clusters_gdf)

# 560 requests for largest set <5 dollars in credit.



Starting to process 140 clusters...

Processing 71/140: grid_id 9945

Original centroid (lat, lon): 1.6952717679950127, 33.146440813501684

Searching at 25 meters radius...
Trying direction search 1/4 at 25m radius
[{'bounds': {'northeast': {'lat': 1.6952692, 'lng': 33.146213}, 'southwest': {'lat': 1.6951201, 'lng': 33.1461323}}, 'copyrights': 'Map data ©2024', 'legs': [{'distance': {'text': '19 m', 'value': 19}, 'duration': {'text': '1 min', 'value': 5}, 'end_address': 'Unnamed Road, Uganda', 'end_location': {'lat': 1.6951201, 'lng': 33.146213}, 'start_address': 'Unnamed Road, Uganda', 'start_location': {'lat': 1.6952692, 'lng': 33.1461323}, 'steps': [{'distance': {'text': '19 m', 'value': 19}, 'duration': {'text': '1 min', 'value': 5}, 'end_location': {'lat': 1.6951201, 'lng': 33.146213}, 'html_instructions': 'Head <b>southeast</b>', 'polyline': {'points': 'mbjIizhiEFEPGBA'}, 'start_location': {'lat': 1.6952692, 'lng': 33.1461323}, 'travel_mode': 'DRIVING'}], 'traffic_speed_entry': [

In [5]:
#now for aisa (in batches to check API cost)

In [6]:
results_aisa = process_all_clusters(aisa_sampled_clusters_gdf)


Starting to process 70 clusters...

Processing 1/70: grid_id 21965

Original centroid (lat, lon): 1.2427973838343365, 33.78459505434681

Searching at 25 meters radius...
Trying direction search 1/4 at 25m radius


[{'bounds': {'northeast': {'lat': 1.2439385, 'lng': 33.78216340000001}, 'southwest': {'lat': 1.2439385, 'lng': 33.78216340000001}}, 'copyrights': 'Map data ©2024', 'legs': [{'distance': {'text': '1 m', 'value': 0}, 'duration': {'text': '1 min', 'value': 0}, 'end_address': 'Unnamed Road, Uganda', 'end_location': {'lat': 1.2439385, 'lng': 33.78216340000001}, 'start_address': 'Unnamed Road, Uganda', 'start_location': {'lat': 1.2439385, 'lng': 33.78216340000001}, 'steps': [{'distance': {'text': '1 m', 'value': 0}, 'duration': {'text': '1 min', 'value': 0}, 'end_location': {'lat': 1.2439385, 'lng': 33.78216340000001}, 'html_instructions': 'Head', 'polyline': {'points': 's}qFoaemE'}, 'start_location': {'lat': 1.2439385, 'lng': 33.78216340000001}, 'travel_mode': 'DRIVING'}], 'traffic_speed_entry': [], 'via_waypoint': []}], 'overview_polyline': {'points': 's}qFoaemE'}, 'summary': '', 'warnings': [], 'waypoint_order': []}]
Found road point at distance: 298.6m
Trying direction search 2/4 at 25m 

In [7]:
#check results 
print(results_dwanwana_dokolo.head())
print(results_aisa.head(5))

   grid_id station_name  buffer_km  est_population_2020  centroid_lat  \
0     9945  Dwanwana FM       25.0           285.974884      1.695272   
1    10457  Dwanwana FM       25.0           206.037659      1.713364   
2    11988  Dwanwana FM       25.0           293.460968      1.722402   
3    11139  Dwanwana FM       25.0           140.694595      1.731455   
4    11991  Dwanwana FM       25.0           174.438141      1.749543   

   centroid_lon                                 centroid_maps_link  \
0     33.146441  https://www.google.com/maps?q=1.69527176799501...   
1     33.173414  https://www.google.com/maps?q=1.71336390995242...   
2     33.254332  https://www.google.com/maps?q=1.72240196467175...   
3     33.209379  https://www.google.com/maps?q=1.73145463704553...   
4     33.254335  https://www.google.com/maps?q=1.74954338634917...   

  cluster_type  nearest_road_lat  nearest_road_lon  distance_to_road  \
0         main          1.695232         33.146160         31.576369

In [15]:

print("aisa", results_aisa.processing_success.value_counts())
#only one missing (which is a replacement cluster, lets just give the centroid)

print("dwanwana_dokolo", results_dwanwana_dokolo.processing_success.value_counts())

#save. 
#results_aisa.to_csv(TEMP_DATA_DIR / 'aisa_roadpoints.csv')      
#results_dwanwana_dokolo.to_csv(TEMP_DATA_DIR / 'dwanwana_dokolo_roadpoints.csv')      
      

#create a roadpoints dataset 
roadpoints_df=pd.concat([results_aisa, results_dwanwana_dokolo])


print(roadpoints_df.station_name.value_counts(dropna=False)) #seems legit
print(roadpoints_df.shape)


aisa processing_success
True    70
Name: count, dtype: int64
dwanwana_dokolo processing_success
True    140
Name: count, dtype: int64
station_name
Aisa FM        70
Dwanwana FM    70
Dokolo FM      70
Name: count, dtype: int64
(210, 15)


In [9]:
#add adress data for the nearest roadpoints using nomatim/reverse geocoding. 
def get_location_details(lat, lon):
    """
    Get administrative areas/adresses using OpenStreetMap/Nominatim
    """
    geolocator = Nominatim(user_agent="my_app")
    
    try:
        # Make the reverse geocoding request
        location = geolocator.reverse((lat, lon), exactly_one=True)
        if not location:
            return None
            
        # Extract address components
        address = location.raw['address']
        location_data = {'nearest_address_full': location.raw['display_name'],
            'village': address.get('village'),
             'district': address.get('state') or address.get('district'),
             'region': address.get('region'),
             'country': address.get('country'),
             'nearest_village_lat': location.raw['lat'],
             'nearest_village_lon': location.raw['lon'],
         }
        
        return location_data
        
    except Exception as e:
        print(f"Error: {e}")
        return None


In [ ]:

#apply to dataframe
results=[]
for index, row in roadpoints_df.iterrows():
    location_data = get_location_details(row['nearest_road_lat'], row['nearest_road_lon'])
    if location_data:
        location_data['grid_id'] = row['grid_id'] #parse the grid id from the original df for indexing/merging. 
        location_data['station_name'] = row['station_name'] #parse the grid id from the original df for indexing/merging. 

        print(location_data)
        results.append(location_data)
        time.sleep(0.01)  
        

{'nearest_address_full': 'Komolo -manga, Pallisa, Eastern Region, Uganda', 'village': 'Komolo -manga', 'district': 'Pallisa', 'region': 'Eastern Region', 'country': 'Uganda', 'nearest_village_lat': '1.2473019', 'nearest_village_lon': '33.7806277', 'grid_id': 21965, 'station_name': 'Aisa FM'}
{'nearest_address_full': 'Kachede, Pallisa, Eastern Region, Uganda', 'village': 'Kachede', 'district': 'Pallisa', 'region': 'Eastern Region', 'country': 'Uganda', 'nearest_village_lat': '1.2520556960898883', 'nearest_village_lon': '33.795196636310806', 'grid_id': 22136, 'station_name': 'Aisa FM'}
{'nearest_address_full': 'Kapuwai, Pallisa, Eastern Region, Uganda', 'village': 'Kapuwai', 'district': 'Pallisa', 'region': 'Eastern Region', 'country': 'Uganda', 'nearest_village_lat': '1.262748771751475', 'nearest_village_lon': '33.76613591556418', 'grid_id': 21627, 'station_name': 'Aisa FM'}
{'nearest_address_full': 'Odusai, Pallisa, Eastern Region, Uganda', 'village': 'Odusai', 'district': 'Pallisa', '

In [18]:
#throw in dataframe
results_df=pd.DataFrame(results)
#how many districts
print(results_df.district.value_counts(dropna=False))
#temp outfile save
results_df.to_csv(TEMP_DATA_DIR/'reversegeocod.csv')


roadpoints_address_df=roadpoints_df.merge(results_df, on=['station_name', 'grid_id'], how='left')
# roadpoints_address_df
roadpoints_address_df.shape

district
Dokolo           61
Kaberamaido      34
Kalaki           29
Kumi             27
Ngora            22
Pallisa          12
Lira District     7
Alebtong          5
Soroti            4
Serere            3
Bukedea           1
Kwania            1
Name: count, dtype: int64


(210, 22)

In [ ]:
print(roadpoints_address_df.village.value_counts(dropna=False))

#12 clusters where we could not find a nearest village, but they mostly have some other marker (e.g. like a school on open streetmap)
#Calcululate the distance to the nearest village to get an idea of how reliable this is? 

def calculate_distance(row):
    '''Calculates distance in km between 2 coordinates, returns None if coordinates are invalid'''
    try:
        point_road = (row['nearest_road_lat'], row['nearest_road_lon'])
        point_village = (row['nearest_village_lat'], row['nearest_village_lon'])
        return geodesic(point_road, point_village).kilometers
    except (ValueError, TypeError):
        return None



roadpoints_address_df['dist_point_to_nearest_address_km'] = roadpoints_address_df.apply(calculate_distance, axis=1)
# #bit of clean-up







village
None                12
Kapuwai              4
NaN                  4
Kabura               3
Ayito (adwoki)       3
                    ..
Acanpii A            1
Oriocudi             1
Barayom (apenyo)     1
Aweikoko             1
Anapakori            1
Name: count, Length: 159, dtype: int64


,grid_id,station_name,buffer_km,est_population_2020,centroid_lat,centroid_lon,centroid_maps_link,cluster_type,nearest_road_lat,nearest_road_lon,...,processing_success,geometry_grid_cell,nearest_address_full,village,district,region,country,nearest_village_lat,nearest_village_lon,dist_point_to_nearest_address_km
0,21965,Aisa FM,25.0,412.028381,1.242797,33.784595,https://www.google.com/maps?q=1.24279738383433...,main,1.243939,33.782163,...,True,POLYGON ((33.789087720369345 1.238272846674644...,"Komolo -manga, Pallisa, Eastern Region, Uganda",Komolo -manga,Pallisa,Eastern Region,Uganda,1.2473019,33.7806277,0.409300
1,22136,Aisa FM,25.0,867.115967,1.251841,33.793586,https://www.google.com/maps?q=1.25184107840705...,main,1.252325,33.794062,...,True,"POLYGON ((33.79807840053194 1.24731652616738, ...","Kachede, Pallisa, Eastern Region, Uganda",Kachede,Pallisa,Eastern Region,Uganda,1.2520556960898883,33.795196636310806,0.129720
2,21627,Aisa FM,25.0,1342.766846,1.260896,33.766624,https://www.google.com/maps?q=1.26089560634678...,replacement,1.259437,33.766239,...,True,POLYGON ((33.77111699937585 1.2563710620307793...,"Kapuwai, Pallisa, Eastern Region, Uganda",Kapuwai,Pallisa,Eastern Region,Uganda,1.262748771751475,33.76613591556418,0.366433
3,20438,Aisa FM,25.0,1109.628174,1.269960,33.703710,https://www.google.com/maps?q=1.26996006966729...,main,1.270047,33.703725,...,True,"POLYGON ((33.70820246722493 1.265435561661601,...","Odusai, Pallisa, Eastern Region, Uganda",Odusai,Pallisa,Eastern Region,Uganda,1.2697977344875335,33.70383628932957,0.030163
4,21119,Aisa FM,25.0,802.255127,1.278996,33.739665,https://www.google.com/maps?q=1.27899647711486...,replacement,1.277351,33.738072,...,True,POLYGON ((33.74415784726423 1.2744719335032875...,"Komolo Kobwin, Pallisa, Eastern Region, Uganda",Komolo Kobwin,Pallisa,Eastern Region,Uganda,1.2773184575182912,33.7380350760635,0.005483
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,9481,Dokolo FM,25.0,488.350555,2.111445,33.119498,https://www.google.com/maps?q=2.11144452049034...,replacement,2.111613,33.118540,...,True,POLYGON ((33.12399361252331 2.1069205986189536...,"Acanpii A, Agali, Lira District, Northern Regi...",Acanpii A,Lira District,None,Uganda,2.112704709518338,33.118102047038526,0.130134
206,9312,Dokolo FM,25.0,304.447693,2.120492,33.110505,https://www.google.com/maps?q=2.12049236240166...,replacement,2.119180,33.109078,...,True,POLYGON ((33.115001510876404 2.115968464354523...,"Oriocudi, Agali, Lira District, Northern Regio...",Oriocudi,Lira District,None,Uganda,2.1243644397356505,33.11049015254601,0.594420
207,11348,Dokolo FM,25.0,424.834106,2.084292,33.218414,https://www.google.com/maps?q=2.08429233098936...,replacement,2.084239,33.217747,...,True,POLYGON ((33.222909959580846 2.079768151713362...,"Barayom (apenyo), Dokolo, Northern Region, Uganda",Barayom (apenyo),Dokolo,None,Uganda,2.0822343,33.2183112,0.230331
208,11690,Dokolo FM,25.0,219.438843,2.102384,33.236402,https://www.google.com/maps?q=2.10238394890964...,main,2.101423,33.236623,...,True,POLYGON ((33.24089783062235 2.0978597189140844...,"Aweikoko, Alebtong, Northern Region, Uganda",Aweikoko,Alebtong,None,Uganda,2.0983261,33.2318774,0.629234


In [23]:
roadpoints_address_df.columns

Index(['grid_id', 'station_name', 'buffer_km', 'est_population_2020',
       'centroid_lat', 'centroid_lon', 'centroid_maps_link', 'cluster_type',
       'nearest_road_lat', 'nearest_road_lon', 'distance_to_road',
       'nearest_road_maps_link', 'found_at_radius', 'processing_success',
       'geometry_grid_cell', 'nearest_address_full', 'village', 'district',
       'region', 'country', 'nearest_village_lat', 'nearest_village_lon',
       'dist_point_to_nearest_address_km'],
      dtype='object')

In [36]:
from shapely import wkt
#create a geo column. 
_roadpoints_address_df=roadpoints_address_df
_roadpoints_address_df['geometry'] = roadpoints_address_df['geometry_grid_cell'].apply(wkt.loads)
_roadpoints_address_df=_roadpoints_address_df.drop(columns='geometry_grid_cell')
enumeration_area_data=gpd.GeoDataFrame(_roadpoints_address_df, geometry='geometry', crs=sampled_clusters_gdf.crs)


In [ ]:
output_path=PROCESSED_DATA_DIR/'enumeration_area_data.gpkg'
crs=sampled_clusters_gdf.crs

enumeration_area_data.to_file(output_path, driver='GPKG', layer='grid_cells') #grid_cells contains the whole dataset. 
centroid_gdf = gpd.GeoDataFrame(
        enumeration_area_data[['grid_id', 'station_name', 'cluster_type',  'centroid_lat', 'centroid_lon', 'centroid_maps_link']],
        geometry=gpd.points_from_xy(
            enumeration_area_data['centroid_lon'],
            enumeration_area_data['centroid_lat']
        ),
        crs=crs
    )
centroid_gdf.to_file(output_path, driver='GPKG', layer='centroids')

    # Create and save road points layer
road_points = gpd.GeoDataFrame(
        enumeration_area_data[['grid_id', 'station_name', 'nearest_road_lat', 'nearest_road_lon', 'distance_to_road', 'nearest_road_maps_link']],
        geometry=gpd.points_from_xy(
            enumeration_area_data['nearest_road_lon'],
            enumeration_area_data['nearest_road_lat']
        ),
        crs=crs
    )

road_points.to_file(output_path, driver='GPKG', layer='road_points')
    
    # Create and nearest village points layer
nearest_village_gdf = gpd.GeoDataFrame(
        enumeration_area_data[['grid_id', 'station_name','nearest_address_full', 'village', 'district',
       'region', 'country', 'nearest_village_lat', 'nearest_village_lon',
       'dist_point_to_nearest_address_km']],
        geometry=gpd.points_from_xy(
            enumeration_area_data['nearest_village_lon'],
            enumeration_area_data['nearest_village_lat']
        ),
        crs=crs
    )

nearest_village_gdf.to_file(output_path, driver='GPKG', layer='village_points')
    


In [ ]:
#check how the geopackage works with multiple layers. 

#test=gpd.read_file(PROCESSED_DATA_DIR/'enumeration_area_data.gpkg')
#test

In [ ]:
# def create_enumeration_area_map(row):
#     """
#     Create a map for a single enumeration area showing:
#     1. The nearest road point
#     2. The grid cell geometry
#     3. The centroid
    
#     Parameters:
#     row: A single row from the results DataFrame
#     """
#     import folium
#     from shapely import wkt
    
#     # Convert WKT string back to geometry
#     grid_geometry = wkt.loads(row.geometry)
    
#     # Create a map centered on the nearest road point (if it exists) or centroid
#     if row.nearest_road_lat and row.nearest_road_lon:
#         center_lat, center_lon = row.nearest_road_lat, row.nearest_road_lon
#     else:
#         center_lat, center_lon = row.centroid_lat, row.centroid_lon
        
#     m = folium.Map(
#         location=[center_lat, center_lon],
#         zoom_start=17,
#         tiles='https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
#         attr='Google Maps'
#     )
    
#     # Add the grid cell geometry
#     folium.GeoJson(
#         grid_geometry.__geo_interface__,
#         style_function=lambda x: {
#             'fillColor': '#ffaf00',
#             'color': '#ff0000',
#             'weight': 2,
#             'fillOpacity': 0.1
#         },
#         popup=f"Grid ID: {row.grid_id}"
#     ).add_to(m)
    
#     # Add the centroid point
#     folium.CircleMarker(
#         location=[row.centroid_lat, row.centroid_lon],
#         radius=6,
#         color='blue',
#         fill=True,
#         popup='Centroid'
#     ).add_to(m)
    
#     # Add the nearest road point if it exists
#     if row.nearest_road_lat and row.nearest_road_lon:
#         folium.CircleMarker(
#             location=[row.nearest_road_lat, row.nearest_road_lon],
#             radius=6,
#             color='red',
#             fill=True,
#             popup=f'Nearest Road Point (Distance: {row.distance_to_road:.1f}m)'
#         ).add_to(m)
        
#         # Add a line connecting centroid to nearest road point
#         folium.PolyLine(
#             locations=[
#                 [row.centroid_lat, row.centroid_lon],
#                 [row.nearest_road_lat, row.nearest_road_lon]
#             ],
#             weight=2,
#             color='red',
#             opacity=0.8
#         ).add_to(m)
    
#     # Add a legend
#     legend_html = '''
#     <div style="position: fixed; bottom: 50px; left: 50px; z-index: 1000; 
#          background-color: white; padding: 10px; border: 2px solid grey; border-radius: 5px;">
#         <p><i style="background: red; width: 10px; height: 10px; display: inline-block; border-radius: 50%;"></i> Nearest Road</p>
#         <p><i style="background: blue; width: 10px; height: 10px; display: inline-block; border-radius: 50%;"></i> Centroid</p>
#         <p><i style="border: 2px solid red; width: 10px; height: 10px; display: inline-block;"></i> Grid Boundary</p>
#     </div>
#     '''
#     m.get_root().html.add_child(folium.Element(legend_html))
    
#     return m

# # Create maps for all enumeration areas
# for idx, row in results_df.iterrows():
#     m = create_enumeration_area_map(row)
#     m.save(f'enumeration_area_map_{row.grid_id}.html')

# # Or test with just one area
# test_map = create_enumeration_area_map(results_df.iloc[0])
# test_map.save('test_enumeration_area_map.html')